In [ ]:
import pandas as pd
import numpy as np
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Load the data from the CSV file
df = pd.read_csv("data.csv")

In [ ]:
# Convert date columns to a datetime object
df['Date'] = pd.to_datetime(df[['Day', 'Month', 'Year']])
df.set_index('Date', inplace=True)

In [ ]:
# Select the crops for prediction
crops = ["Rice", "Wheat", "Atta (Wheat)", "Gram Dal"]

In [ ]:
# Create a dictionary to store the last known data for each state
last_known_data = {}
for state in df['States/UTs'].unique():
    if state not in ['Average Price', 'Maximum Price', 'Minimum Price', 'Modal Price']:
        last_known_data[state] = df[df['States/UTs'] == state].iloc[-1][crops].values

In [ ]:
# Prepare data for LSTM
data = df[crops].values

In [ ]:
# Scale the data to be between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[0:train_size,:]
test_data = scaled_data[train_size:len(scaled_data),:]

In [ ]:
# Create the function to create the dataset
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        X.append(a)
        Y.append(dataset[i + look_back, :])
    return np.array(X), np.array(Y)

In [ ]:
# Set look_back period (number of previous days to consider)
look_back = 7
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)

In [ ]:
# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

In [ ]:
# Create and fit the LSTM network
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(X_train.shape[2]))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
1048/1048 - 11s - 10ms/step - loss: 0.0228
Epoch 2/100
1048/1048 - 10s - 9ms/step - loss: 0.0177
Epoch 3/100
1048/1048 - 8s - 8ms/step - loss: 0.0174
Epoch 4/100
1048/1048 - 6s - 6ms/step - loss: 0.0168
Epoch 5/100
1048/1048 - 4s - 4ms/step - loss: 0.0166
Epoch 6/100
1048/1048 - 4s - 4ms/step - loss: 0.0155
Epoch 7/100
1048/1048 - 5s - 5ms/step - loss: 0.0133
Epoch 8/100
1048/1048 - 4s - 4ms/step - loss: 0.0098
Epoch 9/100
1048/1048 - 6s - 6ms/step - loss: 0.0089
Epoch 10/100
1048/1048 - 4s - 4ms/step - loss: 0.0081
Epoch 11/100
1048/1048 - 5s - 4ms/step - loss: 0.0077
Epoch 12/100
1048/1048 - 5s - 5ms/step - loss: 0.0071
Epoch 13/100
1048/1048 - 5s - 4ms/step - loss: 0.0069
Epoch 14/100
1048/1048 - 4s - 4ms/step - loss: 0.0063
Epoch 15/100
1048/1048 - 6s - 6ms/step - loss: 0.0060
Epoch 16/100
1048/1048 - 9s - 8ms/step - loss: 0.0058
Epoch 17/100
1048/1048 - 6s - 6ms/step - loss: 0.0055
Epoch 18/100
1048/1048 - 4s - 4ms/step - loss: 0.0052
Epoch 19/100
1048/1048 - 6s - 6ms/

In [ ]:
# Make prediction for 6/9/2024
# First, prepare the input data for 6/9/2024 prediction
last_week_data = df[crops].iloc[-look_back:].values
scaled_last_week = scaler.transform(last_week_data)
reshaped_last_week = np.reshape(scaled_last_week, (1, look_back, len(crops)))


In [ ]:
# Predict
prediction_scaled = model.predict(reshaped_last_week)
prediction = scaler.inverse_transform(prediction_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step


In [ ]:
# Create DataFrame for 6/9/2024 predictions
predictions_6_9_2024 = pd.DataFrame(data=prediction, columns=crops, index=['Average Price'])

In [ ]:
# Predict for all states using the last known data
for state, state_data in last_known_data.items():
    scaled_state_data = scaler.transform(state_data.reshape(1,-1))
    reshaped_state_data = np.reshape(scaled_state_data, (1, 1, len(crops)))

    state_prediction_scaled = model.predict(reshaped_state_data)
    state_prediction = scaler.inverse_transform(state_prediction_scaled)

    predictions_6_9_2024.loc[state] = state_prediction[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━

In [ ]:
# Add other metrics (Minimum, Maximum, Modal) based on 6/9/2024 'Average Price' prediction
# Note: Modal price prediction is simplistic and could be improved with a more robust method
predictions_6_9_2024.loc['Maximum Price'] = predictions_6_9_2024.loc['Average Price'] * 1.1
predictions_6_9_2024.loc['Minimum Price'] = predictions_6_9_2024.loc['Average Price'] * 0.9
predictions_6_9_2024.loc['Modal Price']  = predictions_6_9_2024.loc['Average Price'].round()


In [ ]:
# Display the predictions
print(predictions_6_9_2024)


                          Rice      Wheat  Atta (Wheat)   Gram Dal
Average Price        42.070450  30.300486     36.331097  89.694756
Andaman and Nicobar   2.884440  -0.228601      0.208588   7.942561
Andhra Pradesh       -2.836508  -0.052003      3.957927  17.775030
Arunachal pradesh     2.117340   6.298947      9.317301  26.494530
Assam                 9.241463  13.769223     16.490360  33.264790
Bihar                13.237404  16.284227     19.961292  36.120132
Chandigarh           11.239964  14.437267     17.722742  34.635422
Chhattisgarh         10.253281  15.333385     17.676966  33.529385
Delhi                13.657010  16.634432     21.279308  38.063431
DNH and DD            1.486624   5.082497      8.166783  23.483097
Goa                  -2.624865  -2.530104      1.694121  18.687302
Gujarat              10.874998  15.337489     18.080055  34.714699
Haryana              12.517679  15.924286     19.315996  35.260696
Himachal Pradesh     13.845942  14.788275     21.217209  35.03

In [ ]:
model.save('crop_price_lstm_model.h5')

In [ ]:
model = tensorflow.keras.models.load_model('crop_price_lstm_model.h5')